In [34]:
import pandas as pd, numpy as np
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report, roc_curve, f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import time


In [35]:
train_df = pd.read_csv("data/twitter_data_train_multiclass.csv")
test_df = pd.read_csv("data/twitter_data_test_multiclass.csv")

print(train_df.shape)
print(test_df.shape)

(9446, 238)
(1667, 238)


In [36]:
to_drop = [
    'Unnamed: 0',
    'account_type', 
    'account_type_multi',
    'id', 
    'name', 
    'screen_name',
    'profile_image_url',
    'protected',
    'verified',
    'description',
    'description_processed',
    'tweets_list',
    'tweets_list_processed',
    'url',
    'profile_use_background_image', 
    'profile_background_tile'
    ]

scaler = MinMaxScaler() 
train_df[['statuses_count', 'favourites_count', 'followers_count', 'friends_count', 'number_of_mentions', 'listed_count', 'number_of_tags']] = scaler.fit_transform(train_df[['statuses_count', 'favourites_count', 'followers_count', 'friends_count', 'number_of_mentions', 'listed_count', 'number_of_tags']])
test_df[['statuses_count', 'favourites_count', 'followers_count', 'friends_count', 'number_of_mentions', 'listed_count', 'number_of_tags']] = scaler.transform(test_df[['statuses_count', 'favourites_count', 'followers_count', 'friends_count', 'number_of_mentions', 'listed_count', 'number_of_tags']])

X_train, y_train = train_df.drop(to_drop, axis=1), train_df['account_type_multi']
X_test, y_test = test_df.drop(to_drop, axis=1), test_df['account_type_multi']

# validation split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15)

In [37]:
from keras.utils import to_categorical
y_train_cat = to_categorical(y_train)
y_val_cat = to_categorical(y_val)
y_test_cat  = to_categorical(y_test)

In [38]:
start_time = time.time()

model = Sequential()

model.add(Dense(100, activation='relu', input_dim=222))
model.add(Dropout(.15))
model.add(Dense(500, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(6, activation='softmax'))

model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])

model.fit(X_train, y_train_cat, epochs=20, validation_data=(X_val, y_val_cat))

time_taken = time.time() - start_time
print("Total time taken for the program execution", time_taken) # seconds


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 100)               22300     
                                                                 
 dropout_4 (Dropout)         (None, 100)               0         
                                                                 
 dense_17 (Dense)            (None, 500)               50500     
                                                                 
 dense_18 (Dense)            (None, 500)               250500    
                                                                 
 dense_19 (Dense)            (None, 6)                 3006      
                                                                 
Total params: 326,306
Trainable params: 326,306
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
251/251 [============================

Train

In [39]:
best_clf = model

start_time = time.time()

y_pred_train = best_clf.predict(X_train)
y_pred_train = np.argmax(y_pred_train,axis=1)

time_taken = time.time() - start_time
print("Total time taken for the program execution", time_taken) # seconds
print(classification_report(y_train, y_pred_train, digits=5))
# roc_auc_score(y_test, y_pred_test)
print("F1 weighted: ", f1_score(y_train, y_pred_train, average='weighted'))

251/251 [==============================] - 0s 1ms/step
Total time taken for the program execution 0.4391469955444336
              precision    recall  f1-score   support

           0    0.97971   0.99497   0.98728      1990
           1    0.99348   0.99477   0.99412      2296
           2    1.00000   0.96063   0.97992       254
           3    0.99968   0.99237   0.99601      3147
           4    0.98175   0.98535   0.98355       273
           5    1.00000   0.97101   0.98529        69

    accuracy                        0.99228      8029
   macro avg    0.99244   0.98319   0.98770      8029
weighted avg    0.99236   0.99228   0.99228      8029

F1 weighted:  0.9922846145465879


test

In [40]:
start_time = time.time()

y_pred_test = best_clf.predict(X_test)
y_pred_test = np.argmax(y_pred_test,axis=1)

time_taken = time.time() - start_time
print("Total time taken for the program execution", time_taken) # seconds
print(classification_report(y_test, y_pred_test, digits=5))
# roc_auc_score(y_test, y_pred_test)
print("F1 weighted: ", f1_score(y_test, y_pred_test, average='weighted'))

53/53 [==============================] - 0s 937us/step
Total time taken for the program execution 0.12578892707824707
              precision    recall  f1-score   support

           0    0.95704   0.97094   0.96394       413
           1    0.98495   0.99565   0.99027       460
           2    0.96296   0.89655   0.92857        58
           3    0.99543   0.98198   0.98866       666
           4    0.94643   1.00000   0.97248        53
           5    1.00000   0.94118   0.96970        17

    accuracy                        0.98020      1667
   macro avg    0.97447   0.96438   0.96894      1667
weighted avg    0.98039   0.98020   0.98018      1667

F1 weighted:  0.9801828150802312
